## Set up paths and imports

In [1]:
import os

import torch
from torchvision import transforms

if not os.path.exists("./notebooks"):
    %cd ..

import src.model
from src.training import train, validate
from src.dataset import prepare_dataset_loaders
from src.data_processing import load_mean_std
from src.config import DATASET_DIR

wandb_enabled = False

/Users/boryskurdek/Studia/ml/iml


## 1. Load standarization data and define Config

In [2]:
mean, std = load_mean_std(f"{DATASET_DIR}/scaling_params.json")

class Config:
    def __init__(self, lr=0.001, epochs=40, batch_size=32):
        self.learning_rate = lr
        self.epochs = epochs
        self.batch_size = batch_size

### Optionally initialize W&B project

In [6]:
import wandb

wandb_enabled = True

## 2. Define training and validation loop

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def do_train(name, train_loader, val_loader, config, model, criterion, optimizer):
    if wandb_enabled:
            wandb.init(name=name, project="iml", config=vars(config))
 
    model.device = device
    model.to(device)

    for epoch in range(config.epochs):
        print(f"Epoch {epoch+1}/{config.epochs}")

        if wandb_enabled:
            logger = wandb.log
        else:
            logger = lambda data,step: print(f"  Step {step}: {data}")

        train(model, train_loader, criterion, optimizer, epoch, logger, len(train_loader) // 5 - 1)
        metrics = validate(model, val_loader)
        print(metrics)

        if wandb_enabled:
            wandb.log({"validation/recall": metrics.recall, "validation/accuracy": metrics.accuracy, "validation/precision": metrics.precision, "validation/f1": metrics.f1, "epoch": epoch+1})

    model_path = f"./models/{name}.pth"
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    torch.save(model.state_dict(), model_path)

    if wandb_enabled: 
        wandb.save(model_path)
        wandb.finish()
    


In [8]:
name = "TutorialCNN without standardization"
model = src.model.TutorialCNN()
config = Config()
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
])
train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, config.batch_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
   
do_train(name, train_loader, val_loader, config, model, criterion, optimizer)

Epoch 1/40
  Step 19: {'train/accuracy': 0.7, 'train/loss': 0.5896340206265449}
  Step 39: {'train/accuracy': 0.728125, 'train/loss': 0.5832896664738655}
  Step 59: {'train/accuracy': 0.7328125, 'train/loss': 0.5783891782164574}
  Step 79: {'train/accuracy': 0.7515625, 'train/loss': 0.5569380313158036}
  Step 99: {'train/accuracy': 0.70625, 'train/loss': 0.6015354737639427}
Metrics:
    F1: 0.82,
    Accuracy: 0.69,
    Recall: 1.00,
    Precision: 0.69,
    False acceptance: 1.00,
    False rejection: 0.00
Epoch 2/40
  Step 125: {'train/accuracy': 0.7390625, 'train/loss': 0.583763538300991}
  Step 145: {'train/accuracy': 0.7078125, 'train/loss': 0.6105416715145111}
  Step 165: {'train/accuracy': 0.73125, 'train/loss': 0.5765310242772103}
  Step 185: {'train/accuracy': 0.740625, 'train/loss': 0.5720834374427796}
  Step 205: {'train/accuracy': 0.7359375, 'train/loss': 0.5755416110157967}
Metrics:
    F1: 0.82,
    Accuracy: 0.69,
    Recall: 1.00,
    Precision: 0.69,
    False acceptan

KeyboardInterrupt: 

In [ ]:
name = "TutorialCNN"
model = src.model.TutorialCNN()
config = Config()
transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, config.batch_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

do_train(name, train_loader, val_loader, config, model, criterion, optimizer)

In [5]:
name = "OriginalSizeCNN"
model = src.model.OriginalSizeCNN()
config = Config()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, config.batch_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

do_train(name, train_loader, val_loader, config, model, criterion, optimizer)

Epoch 1/40
  Step 19: {'train/accuracy': 0.75, 'train/loss': 0.586673055589199}
  Step 39: {'train/accuracy': 0.721875, 'train/loss': 0.5975300416350364}
  Step 59: {'train/accuracy': 0.7125, 'train/loss': 0.5834112375974655}
  Step 79: {'train/accuracy': 0.7640625, 'train/loss': 0.5202491581439972}
  Step 99: {'train/accuracy': 0.8265625, 'train/loss': 0.40197805389761926}
Metrics:
    F1: 0.87,
    Accuracy: 0.81,
    Recall: 0.97,
    Precision: 0.80,
    False acceptance: 0.55,
    False rejection: 0.03
Epoch 2/40
  Step 125: {'train/accuracy': 0.89375, 'train/loss': 0.2637752965092659}
  Step 145: {'train/accuracy': 0.8828125, 'train/loss': 0.26397902220487596}
  Step 165: {'train/accuracy': 0.9109375, 'train/loss': 0.2517184842377901}
  Step 185: {'train/accuracy': 0.928125, 'train/loss': 0.2056976530700922}
  Step 205: {'train/accuracy': 0.9203125, 'train/loss': 0.18452888764441014}
Metrics:
    F1: 0.94,
    Accuracy: 0.91,
    Recall: 0.93,
    Precision: 0.94,
    False accep

KeyboardInterrupt: 

In [5]:
name = "DropoutCNN"
model = src.model.DropoutCNN()
config = Config()
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, config.batch_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

do_train(name, train_loader, val_loader, config, model, criterion, optimizer)

Epoch 1/40
  Step 19: {'train/accuracy': 0.725, 'train/loss': 0.6869034141302108}
  Step 39: {'train/accuracy': 0.7203125, 'train/loss': 0.5891412854194641}
  Step 59: {'train/accuracy': 0.7390625, 'train/loss': 0.5721561908721924}
  Step 79: {'train/accuracy': 0.709375, 'train/loss': 0.575371490418911}
  Step 99: {'train/accuracy': 0.7375, 'train/loss': 0.5569010183215142}
Metrics:
    F1: 0.82,
    Accuracy: 0.69,
    Recall: 1.00,
    Precision: 0.69,
    False acceptance: 1.00,
    False rejection: 0.00
Epoch 2/40
  Step 125: {'train/accuracy': 0.7171875, 'train/loss': 0.5491385981440544}
  Step 145: {'train/accuracy': 0.7640625, 'train/loss': 0.543464682251215}
  Step 165: {'train/accuracy': 0.7453125, 'train/loss': 0.5226360008120536}
  Step 185: {'train/accuracy': 0.8015625, 'train/loss': 0.4610556498169899}
  Step 205: {'train/accuracy': 0.7765625, 'train/loss': 0.47816406935453415}
Metrics:
    F1: 0.88,
    Accuracy: 0.82,
    Recall: 0.99,
    Precision: 0.80,
    False acce